In [236]:
from edgeml import edgeml
import requests as req
import random
import pandas as pd
import tsfresh

In [237]:
keylocal = "RuW7CkG0keGCn9OH7yG40XY3nZz4Vfdl+gUgvpeiqxOP1eIwCwg90BjMrQVQV5fz3/zK1zBZ4uP+WFelZUmYTw=="
key = "EoXl+XUYu71AQ/g3A2BTE1MS2Lpwi07Ud8BQ6Cm9IDgKV/QaiZN0oLIvF+/1jRAJSPT+HJivHDrtx6xRqg+6xQ=="
startTime = 1618760115000

collector = edgeml.datasetCollector("http://localhost",
                                    keylocal,
                                    "Example Dataset",
                                    False)

labels = [random.randint(1,3) for i in range(50)]
for i in range(50):
    collector.addDataPoint("Acc", random.randint(1,50)/10.0, startTime + i)
    collector.addDataPoint("Mag", random.randint(1,50)/10.0, startTime + i)

collector.onComplete()

In [245]:
url = "https://app.edge-ml.org"
urllocal = "http://localhost"
project = edgeml.getProject(url, key)
datasets = project['datasets']
# print(project)

In [246]:
labelset = {}
labelIds = {'unlabeled': -1}
labelId = 0
for dataset in datasets:
    labels = dataset['labels']
    for labelData in labels:
        for label in labelData:
            name = label['name']
            start = label['start']
            end = label['end']
            if not name in labelset:
                labelset[name] = []
            labelset[name].append((start, end))
            labelIds[name] = labelId
            labelId = labelId + 1

id = 0
values = {'id': []}
dataPointLabels = []
for dataset in datasets:
    sensors = dataset['sensors']
    for sensor in sensors:
        name = sensor['name']
        data = sensor['data']
        if not name in values:
            values[name] = []
        for dataPoint in data:
            timestamp = dataPoint['timestamp']
            dataPointValue = dataPoint['datapoint']
            values[name].append(dataPointValue)
            if sensor == sensors[0]:
                values['id'].append(id)
                id = id + 1
                labeled = False
                for label, intervals in labelset.items():
                    if labeled:
                        break
                    for interval in intervals:
                        start = interval[0]
                        end = interval[1]
                        if timestamp >= start and timestamp <= end:
                            dataPointLabels.append(labelIds[label])
                            labeled = True
                            break
                if not labeled:
                    dataPointLabels.append(-1)

dataFrame = pd.DataFrame(values)
# print(dataPointLabels)

In [240]:
settings = tsfresh.feature_extraction.settings.MinimalFCParameters()
extracted = tsfresh.extract_features(dataFrame, column_id="id", default_fc_parameters=settings)
print(extracted)

Feature Extraction: 100%|██████████| 30/30 [00:03<00:00,  9.88it/s]


      ACC_x__sum_values  ACC_x__median  ACC_x__mean  ACC_x__length  \
0                   0.0            0.0          0.0            1.0   
1                  -8.0           -8.0         -8.0            1.0   
2                  -8.0           -8.0         -8.0            1.0   
3                   2.0            2.0          2.0            1.0   
4                   3.0            3.0          3.0            1.0   
...                 ...            ...          ...            ...   
6540              304.0          304.0        304.0            1.0   
6541              304.0          304.0        304.0            1.0   
6542              304.0          304.0        304.0            1.0   
6543              305.0          305.0        305.0            1.0   
6544              305.0          305.0        305.0            1.0   

      ACC_x__standard_deviation  ACC_x__variance  ACC_x__root_mean_square  \
0                           0.0              0.0                      0.0   
1    

In [257]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(extracted, dataPointLabels, random_state = 5)


In [258]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(x_train)
trans_x_train = scaler.transform(x_train)
trans_x_test = scaler.transform(x_test)
trans_x_train = pd.DataFrame(trans_x_train,columns=x_train.columns)
trans_x_train.describe()

,ACC_x__sum_values,ACC_x__median,ACC_x__mean,ACC_x__length,ACC_x__standard_deviation,ACC_x__variance,ACC_x__root_mean_square,ACC_x__maximum,ACC_x__minimum,ACC_y__sum_values,...,ACC_y__minimum,ACC_z__sum_values,ACC_z__median,ACC_z__mean,ACC_z__length,ACC_z__standard_deviation,ACC_z__variance,ACC_z__root_mean_square,ACC_z__maximum,ACC_z__minimum
count,4908.000000,4908.000000,4908.000000,4908.0,4908.0,4908.0,4908.000000,4908.000000,4908.000000,4908.000000,...,4908.000000,4908.000000,4908.000000,4908.000000,4908.0,4908.0,4908.0,4908.000000,4908.000000,4908.000000
mean,-0.442141,-0.442141,-0.442141,0.0,0.0,0.0,0.434655,-0.442141,-0.442141,0.215963,...,0.215963,-1.148179,-1.148179,-1.148179,0.0,0.0,0.0,-1.112463,-1.148179,-1.148179
std,0.714472,0.714472,0.714472,0.0,0.0,0.0,0.686206,0.714472,0.714472,2.748872,...,2.748872,2.450458,2.450458,2.450458,0.0,0.0,0.0,2.336092,2.450458,2.450458
min,-2.927503,-2.927503,-2.927503,0.0,0.0,0.0,-0.119165,-2.927503,-2.927503,-8.705376,...,-8.705376,-11.269807,-11.269807,-11.269807,0.0,0.0,0.0,-8.753747,-11.269807,-11.269807
25%,-0.972382,-0.972382,-0.972382,0.0,0.0,0.0,-0.081081,-0.972382,-0.972382,-0.349677,...,-0.349677,-0.980728,-0.980728,-0.980728,0.0,0.0,0.0,-0.980728,-0.980728,-0.980728
50%,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
75%,0.027618,0.027618,0.027618,0.0,0.0,0.0,0.918919,0.027618,0.027618,0.650323,...,0.650323,0.019272,0.019272,0.019272,0.0,0.0,0.0,0.019272,0.019272,0.019272
max,3.986191,3.986191,3.986191,0.0,0.0,0.0,4.165848,3.986191,3.986191,6.895484,...,6.895484,9.967880,9.967880,9.967880,0.0,0.0,0.0,9.967880,9.967880,9.967880


In [259]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(trans_x_train,y_train)


RandomForestClassifier()

In [260]:
from sklearn.metrics import accuracy_score
prediction = clf.predict(trans_x_train)
print("accuracy_score train :", accuracy_score(y_train,prediction))

accuracy_score train : 0.9981662591687042
